Trying first solo ml project with sleep data

In [155]:
import pandas as pd
pd.options.mode.chained_assignment = None

data_original = pd.read_csv("data/sleepdata.csv",header="infer",sep=";")
data_original = data_original.drop(["Start", "End", "Wake up"], axis=1)

print(data_original.head())

  Sleep quality Time in bed             Sleep Notes  Heart rate  \
0          100%        8:32                     NaN        59.0   
1            3%        0:16           Stressful day        72.0   
2           98%        8:30                     NaN        57.0   
3           65%        7:32                     NaN         NaN   
4           72%        6:44  Drank coffee:Drank tea        68.0   

   Activity (steps)  
0                 0  
1                 0  
2                 0  
3                 0  
4                 0  


Getting a sample data to work on before working on the main data

In [156]:
data = data_original.sample(frac=0.2, replace=False)
data = data.reset_index()

Changing the time in bed attribute to be more meaningful

In [157]:
print(data)

     index Sleep quality Time in bed  \
0       54           69%        7:25   
1      768           79%        7:33   
2      455           52%        6:33   
3      525           87%        8:09   
4      846           83%        8:01   
..     ...           ...         ...   
172    881           91%        8:24   
173    361           70%        7:28   
174     91           50%        7:34   
175    872           65%        5:23   
176     11           88%        7:16   

                                    Sleep Notes  Heart rate  Activity (steps)  
0                                  Drank coffee        61.0                 0  
1                                           NaN         NaN              6235  
2                          Drank tea:Worked out         NaN              8737  
3                                     Drank tea         NaN              3669  
4                                           NaN         NaN              1334  
..                                     

In [158]:
for index, value in data["Time in bed"].items():
    splitString = value.split(":")
    hours, minutes = splitString[0], splitString[1]
    totalTime = (int(hours) * 60) + int(minutes)
    data["Time in bed"][index] = str(totalTime)

print(data)

     index Sleep quality Time in bed  \
0       54           69%         445   
1      768           79%         453   
2      455           52%         393   
3      525           87%         489   
4      846           83%         481   
..     ...           ...         ...   
172    881           91%         504   
173    361           70%         448   
174     91           50%         454   
175    872           65%         323   
176     11           88%         436   

                                    Sleep Notes  Heart rate  Activity (steps)  
0                                  Drank coffee        61.0                 0  
1                                           NaN         NaN              6235  
2                          Drank tea:Worked out         NaN              8737  
3                                     Drank tea         NaN              3669  
4                                           NaN         NaN              1334  
..                                     

Handle missing data

In [159]:
for col in data.columns:
    print(col, ":", data[col].isna().sum(), data[col].dtype)
    if data[col].dtype == "float64":
        data[col] = data[col].fillna(data[col].median())

index : 0 int64
Sleep quality : 0 object
Time in bed : 0 object
Sleep Notes : 50 object
Heart rate : 145 float64
Activity (steps) : 0 int64


In [162]:
from sklearn import tree

Y = data['Sleep quality']
X = data.drop(["Sleep quality", "Heart rate", "Sleep Notes"], axis=1)

clf = tree.DecisionTreeClassifier(criterion="entropy", max_depth=10)
clf = clf.fit(X,Y)

In [164]:
import pydotplus 
from IPython.display import Image

dot_data = tree.export_graphviz(clf, feature_names=X.columns, filled=True, 
                                out_file=None) # generates a GraphViz representation of the decision tree
graph = pydotplus.graph_from_dot_data(dot_data) 
Image(graph.create_png())

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.72822 to fit

